In [4]:
from brightpath import SimaproConverter
fp="/Users/romain/Documents/ESA_LCA_DB_1.2.0f_e3.9.1_-_EXTERNAL_csv_original.CSV"
fp = "/Users/romain/GitHub/brightpath/dev/simapro_13-06-2023.csv"

In [5]:
c = SimaproConverter(
    filepath=fp,
)

ValueError: math domain error

In [3]:
c.convert_to_brightway()

- format exchanges
- remove empty datasets
- remove empty exchanges
- check inventories
Done!


In [4]:
import bw2data
bw2data.projects.set_current("simapro")

In [5]:
c.i.match_database(fields=["name", "reference product", "location"])
c.i.match_database("ecoinvent 3.9.1 cutoff" ,fields=["name", "reference product", "location"])
c.i.match_database("biosphere3" ,fields=["name", "categories"])


Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


In [6]:
c.i.statistics()

774 datasets
31787 exchanges
2661 unlinked exchanges
  Type biosphere: 480 unique unlinked exchanges
  Type technosphere: 14 unique unlinked exchanges


(774, 31787, 2661)

In [7]:
for x in list(c.i.unlinked):
    if x["type"] == "technosphere":
        print(x["name"], x["reference product"], x["location"])

recycling of pe PE (waste treatment) GLO
recycling of steel and iron Steel and iron (waste treatment) GLO
recycling of aluminium Aluminium (waste treatment) GLO
dross for recycling Dross for recycling GLO
refractory Refractory GLO
dust, unspecified Dust, unspecified GLO
waste water - untreated, slightly organic and anorganic contaminated eu-27 s Waste water - untreated, slightly organic and anorganic contaminated EU-27 S GLO
slags Slags GLO
organic carbon, placed in landfill Organic carbon, placed in landfill GLO
waste mass, total, placed in landfill Waste mass, total, placed in landfill GLO
ethene (ethylene), from steam cracking, production mix, at plant, gaseous eu-27 s Ethene (ethylene), from steam cracking, production mix, at plant, gaseous EU-27 S GLO
continuous filament glass fibre (assembled rovings), at plant rer s Continuous filament glass fibre (assembled rovings), at plant RER S GLO
waste water - untreated, organic contaminated eu-27 s Waste water - untreated, organic contam

In [8]:
for x in list(c.i.unlinked):
    if x["type"] == "biosphere":
        if x["categories"][0] == "air":
            print(x["name"], x["categories"], )

Mineral oil ('air',)
Aliphatic aldehydes ('air',)
Nitrogen dioxide ('air',)
Hydrogen cyanide ('air',)
Particulates, unspecified ('air',)
Hydrazine, methyl- ('air',)
2-Methyl-4-chlorophenoxyacetic acid ('air', 'non-urban air or from high stacks')
Benzene, dichloro ('air', 'urban air close to ground')
Benzo(a)anthracene ('air',)
Butyric acid, 4-(2,4-dichlorophenoxy)- ('air', 'non-urban air or from high stacks')
Methyl amine ('air',)
Methyl amine ('air', 'urban air close to ground')
Nitrogen monoxide ('air',)
2-Methyl pentane ('air', 'urban air close to ground')
Nickel compounds ('air',)
Benzene, dichloro ('air',)
Hydrazine ('air',)
Hydrazine, 2-hydroxyethyl- ('air',)
Zinc, fume or dust ('air', 'non-urban air or from high stacks')
Oxygen ('air', 'non-urban air or from high stacks')
1-Methyl-2-pyrrolidinone ('air',)
Sodium sulfate ('air',)
aluminum oxide ('air',)
Fatty acid methyl ester ('air',)
Hexamethylene diamine ('air',)
Kaolin ('air',)
Methyl 2,2-dimethylpropanoate ('air',)
Methyl pe

In [9]:
c.i.drop_unlinked(i_am_reckless=True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.02 seconds


In [10]:
if "ESA LCA External 1.2.0f e3.9.1" in bw2data.databases:
    del bw2data.databases["ESA LCA External 1.2.0f e3.9.1"]
c.i.write_database()

Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/15/2024 20:30:44
  Finished: 05/15/2024 20:30:45
  Total time elapsed: 00:00:00
  CPU %: 100.20
  Memory %: 2.78
Created database: ESA LCA External 1.2.0f e3.9.1


Brightway2 SQLiteBackend: ESA LCA External 1.2.0f e3.9.1

In [12]:
bw2data.get_activity(('ESA LCA External 1.2.0f e3.9.1', '096a57a16a8003388f2a16d98392ac2a'))["name"]

'Laser Cutting, Titanium alloys, Nd:YAG laser (processing)'

In [17]:
for ds in c.i.data:
    if ds["name"] == "Laser Cutting, Titanium alloys, Nd:YAG laser (processing)":
        for e in ds["exchanges"]:
            if e["name"].startswith("recycling of titanium"):
                print(e)

{'uncertainty type': 2, 'scale': 0.06551413120320204, 'loc': -10.24316492010629, 'negative': False, 'amount': 3.56e-05, 'categories': ('Waste to treatment',), 'name': 'recycling of titanium', 'unit': 'kilogram', 'comment': '(2,2,1,5,2,na)', 'type': 'technosphere', 'input': ('ESA LCA External 1.2.0f e3.9.1', '3bed37dae7421ef0e122d52a7bf0d64b'), 'product': 'Titanium (waste treatment) (dummy)', 'location': 'GLO', 'reference product': 'Titanium (waste treatment) (dummy)', 'output': ('ESA LCA External 1.2.0f e3.9.1', '096a57a16a8003388f2a16d98392ac2a')}


In [16]:
for ds in c.i.data:
    if ds["name"].startswith("recycling of titanium"):
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("product"), e.get("location"), e["input"][-1])

In [17]:
import bw2calc
lca = bw2calc.LCA({bw2data.Database("ESA LCA External 1.2.0f e3.9.1").random(): 1})

In [18]:
lca.lci()

NonsquareTechnosphere: Technosphere matrix is not square: 22006 activities (columns) and 22008 products (rows). Use LeastSquaresLCA to solve this system, or fix the input data

In [21]:
act, prod, bio = lca.reverse_dict()

In [23]:
lca.technosphere_matrix.shape

(22008, 22006)

In [26]:
lca.product_dict[22008]

17289

In [27]:
bw2data.Database("ESA LCA External 1.2.0f e3.9.1").random().key

('ESA LCA External 1.2.0f e3.9.1', 'c8a2e79fa5780adbb6fc9b7cb2f3b9f0')

In [28]:
acts = [(a["name"], a["reference product"], a["location"]) for a in bw2data.Database("ESA LCA External 1.2.0f e3.9.1")]

In [31]:
prods = []
for ds in bw2data.Database("ESA LCA External 1.2.0f e3.9.1"):
    for e in ds.exchanges():
        if e["type"] == "production":
            prods.append((e["name"], e["product"], e["location"]))
len(prods)

768

In [32]:
[x for x in acts if x not in prods]

[]

In [33]:
[x for x in prods if x not in acts]

[]

In [38]:
code_a = [a["code"] for a in bw2data.Database("ESA LCA External 1.2.0f e3.9.1")]
print(len(code_a))
print(len(list(set(code_a))))

768
768


In [39]:
code_p = []
for ds in bw2data.Database("ESA LCA External 1.2.0f e3.9.1"):
    for e in ds.exchanges():
        if e["type"] == "production":
            code_p.append(e["input"][-1])
print(len(code_p))
print(len(list(set(code_p))))

768
768


In [40]:
[x for x in code_a if x not in code_p]

[]

In [41]:
[x for x in code_p if x not in code_a]

[]

In [45]:
lca.fix_dictionaries()

True

In [47]:
act, prod, bio = lca.reverse_dict()

In [51]:
code_acts = list(act.values())

In [52]:
for k, v in prod.items():
    if v not in code_acts:
        print(v)

('ESA LCA External 1.2.0f e3.9.1', 'fcaf2adc469a2a2b525e68af29a0b8d5')
('ESA LCA External 1.2.0f e3.9.1', '3bed37dae7421ef0e122d52a7bf0d64b')


In [59]:
for ds in c.i.data:
    if ds["code"] == "3bed37dae7421ef0e122d52a7bf0d64b":
        print(ds["name"])

In [61]:
for ds in bw2data.Database("ESA LCA External 1.2.0f e3.9.1"):
    if ds["code"] == "3bed37dae7421ef0e122d52a7bf0d64b":
        print(ds["name"])

d067d9db2e60e6e58410da2bff825754
52cdfc70bdb717393b98a6f7d142e617
32a681241a2a746d2a5e146a6542d3f4
52ddfab8525295d6ea6699016588affc
32fbc5be8f12d20a4593bfca44da4380
f8fbe3fb3acbffcdb5643ea82b480840
c352a1226ce7ae5ebe8e0370849c0af8
1d196e727734b0e4ccf116652f8dae06
0eec3a37ce3fded0cc126f1270833cbe
d5c532dee94793be5768bbbf1eff93cd
abba572f5185707ac50212dabb615556
a3f79a3bae4978b9d25dd2765baee7dc
770163335bab312febbd07efb90a01ce
be768fe278d55aa712e4a7365a8d359f
0117e119f0b087578ffff0e5074badd7
cdbb802f781e71e3c3dde9525c5f0bcb
62aaa83328c1b2e603821d2c37213bc3
2b3a2b7adcaef03a3b17ecfd262049f4
1eb15127794b0ee987651419462b13c6
701c62d5a2f4eab6059361ab5c351471
ee6003ca89113e7acff683f7c9e96994
0854a576bdabd84867006ad571210d3d
fab5e2f7ad2ab1af5588ddf860b68071
0787bdb615d846a80aee50cdecd3608e
4635c51f9e8e779112758bdfdab7f8fc
d844ed71bec213d2ff0ab7f1793caaeb
84776350e2a6f48db412ebdb7728db82
906e20dc8d160ec3229304638736e80f
08d17a13cc35bc60f4f6d5ee7ed71563
328105d05bde1ca457ec26f4251c6a1e
544dede3d4

In [57]:
bw2data.get_activity(('ESA LCA External 1.2.0f e3.9.1', '3bed37dae7421ef0e122d52a7bf0d64b'))

ActivityDatasetDoesNotExist: <Model: ActivityDataset> instance matching query does not exist:
SQL: SELECT "t1"."id", "t1"."data", "t1"."code", "t1"."database", "t1"."location", "t1"."name", "t1"."product", "t1"."type" FROM "activitydataset" AS "t1" WHERE (("t1"."database" = ?) AND ("t1"."code" = ?)) ORDER BY Random() LIMIT ? OFFSET ?
Params: ['ESA LCA External 1.2.0f e3.9.1', '3bed37dae7421ef0e122d52a7bf0d64b', 1, 0]